In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-b6dl9kyc
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-b6dl9kyc
     |████████████████████████████████| 394.7MB 41kB/s 
  Created wheel for pykospacing: filename=pykospacing-0.4-cp37-none-any.whl size=2255638 sha256=b945d4a5517988b212c2ce2c55de91411390f58956839bcafab1b70c4fe3479b
  Stored in directory: /tmp/pip-ephem-wheel-cache-uhncmcca/wheels/4d/45/58/e26cb2b7f6a063d234158c6fd1e5700f6e15b99d67154340ba
Successfully built pykospacing
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 49.0MB/s 
     |████████████████████████████████| 92kB 11.7MB/s 
     |████████████████████████████████| 460kB 51.7MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
!pip install soynlp

     |████████████████████████████████| 419kB 19.6MB/s 


In [ ]:
!pip install lda

     |████████████████████████████████| 358kB 12.8MB/s 
     |████████████████████████████████| 102kB 10.5MB/s 


In [ ]:
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import *
from konlpy.utils import pprint
import numpy as np
import lda
from sklearn.pipeline import Pipeline
import pandas as pd
import statsmodels.api as sm
from collections import Counter

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import re
import os
import sys
import json

from pykospacing import spacing
from konlpy.tag import Kkma
from konlpy.tag import Okt
import soynlp

kkma = Kkma()
okt = Okt()

In [ ]:
twitter = Twitter()
english = re.compile(r'[a-zA-Z0-9]')
korean = re.compile(r'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]')

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
def preprocessing(review):
    total_review = ''
    #인풋리뷰
    for idx in range(len(review)):
        r = review[idx]
        #하나의 리뷰에서 문장 단위로 자르기
        for sentence in kkma.sentences(r):
            sentence = re.sub('([a-zA-Z])','',sentence)
            sentence = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',sentence)
            sentence = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',sentence)
            if len(sentence) == 0:
                continue
            if len(sentence) < 198:
                sentence = spacing(sentence)
            sentence += '. '
            total_review += sentence
    return total_review

In [47]:
service_good_feature = {'사장':['친절','스윗','센스'],
                        '알바':['친절','스윗','센스'],
                        '직원': ['친절','스윗','센스','전문적'],
                        '일을':['잘','빠르다'],
                        '일도':['잘','빠르다'],
                        '서빙':['잘','빠르다'],
                        '서비스':['좋다'],
                        '대기':[],
                        '영업':[]}

service_bad_feature = {'알바':['불친','똑바로','재수','잘못','실수','짜증','답답','교육','엉망'],
                       '사장':['불친','똑바로','재수','잘못','실수','짜증','답답','엉망'],
                       '직원':['불친','똑바로','재수','잘못','실수','짜증','교육','답답','엉망','들쑥날쑥'],
                      '일을':['못','느리다','답답','잘못','실수'],
                      '일도':['못','느리다', '천천히','잘못','실수','답답'],
                      '서빙':['못','느리다','천천히','답답','잘못','실수'],
                       '서비스':['나쁘다'],
                       '대기':['길다'],
                       '영업':['일찍','짧']}

service_good_emotion = ['스윗', '착하다', '센스', '친절', '빠르다', '감동', '훈훈', '싹싹하다', '미소','꼼꼼','신속','꼼꼼']

service_bad_emotion = ['실수', '엉성', '불친', '개판', '똑바로', '교육', '느리', '느리다', 
                       '컴플레인', '재수', '인상', '짜증', '싸가지', '나쁘', '센스없', '빡치', '잘못', '헬','소진']


atmosphere_good_feature = {'분위기': ['좋다','괜찮','조용','깔끔','적당','깡패','굉장','아담','완벽','아기자기','고급','최고','세련','만족','아늑','훌륭','예쁘','이쁘','짱',
                                   '심쿵','따뜻','깨끗','독특','매력','모던','취향저격','맘','마음','클래식','아름','인상적','귀엽','포근','ㅅㅌㅊ'], 
                           '매장': ['좋다','괜찮','조용','깔끔','적당','깡패','굉장','아담','완벽','아기자기','고급','최고','세련','만족','아늑','훌륭','예쁘','이쁘','짱',
                                    '심쿵','따뜻','깨끗','독특','매력','모던','취향저격','맘','마음','클래식','아름','인상적','귀엽','포근'],
                           '인테리어':['취향','멋지다'],
                           '시설':['좋다'],
                           '홀': ['좋다','괜찮','조용','깔끔','적당','깡패','굉장','아담','완벽','아기자기','고급','최고','세련','만족','아늑','훌륭','예쁘','이쁘','짱',
                                    '심쿵','따뜻','깨끗','독특','매력','모던','취향저격','맘','마음','클래식','아름','인상적','귀엽','포근'],
                           '주차':['지원','무료','넓','가능'],
                           '의자':['많','넉넉'],
                          '자리':['많','넉넉','넓'],
                          '화장실':['편하다','깨끗','쾌적'],
                          '마스크':[],
                          '좌석':[],
                          '콘센트':[],
                          '기분':['좋'],
                          '와이파이':['무료','빠르다','있다','좋다'],
                           '충전':[]}

atmosphere_bad_feature = {'분위기': ['나쁘','나쁜','바쁘','바쁜','어수선','이상','촌스','별로','부담',
                                  '시끄','복잡','ㅎㅌㅊ','더러','더럽','시장','만석','바글','복작','북적','답답'],
                          '매장':['협소','좁다','답답','관리'],
                          '인테리어':[],
                          '시설':[],
                          '홀':['협소','좁다','답답','관리'],
                          '주차':['좁다','불가','협소','유료','촉박','어렵다','비싸','창렬'],
                          '의자':['좁다','딱딱'],
                          '자리':['좁다','딱딱','협소','더럽','엉망'],
                          '화장실':['좁다','딱딱','협소','더럽','엉망','불가','불량'],
                          '마스크':[],
                          '좌석':['적다'],
                          '콘센트':['적다'],
                          '기분':['나쁘','나쁜','나빠','상하','상해','상함','상했'],
                          '와이파이':['느리다','느린','느림','먹통','불편','약하다','끊','최악','쓰레기'],
                          '충전':['부족','불편']
                          }


atmosphere_good_emotion = ['클래식', '세련', '조용', '편하다', '아늑','고급', '이쁘', '쾌적', '여유',
                           '넓다', '모던', '인상적', '취향저격', '맘', '짱', '아기자기', '예쁘', '마음', '완벽', '귀엽',
                           '독특', '깨끗', '따뜻', '깡패', '아름', '매력', '아담', '포근']

atmosphere_bad_emotion = ['바글', '복작', '북적', '만석', '어수선', '불편',
                          '더러', '불량', '복잡', '제재', '바쁜', '시끄', '협소', '더럽', 
                          '바쁘', '답답', '좁', '촌스', '관리', '시장']





taste_good_feature = {'간':['맞','적절','딱','환상','담백'],
                      '커피':['깔끔','고소','부드','좋다','진리','괜찮','끝'],
                      '아메':[],
                      '아아':[],
                      '라떼':['고소','부드','깔끔','좋다'],
                      '라뗴':['고소','부드','깔끔','좋다'],
                      '라테':['부드'],
                      '맛':['있다','좋다','나다','최고','조화'],
                      '케익':['부드','달콤'],
                     '케잌':['부드','달콤'],
                     '케이크':['부드','달콤'],
                     '마카롱':['부드','달콤'],
                      '메뉴':[],
                     '샌드위치':['존맛'],
                     '자몽':['쵝오'],
                      '스콘':[],
                      '돌체':['짱'],
                      '기대':['이상']}

taste_bad_feature = {'간':['세','쎄','강하다','별로'],
                     '커피':['별로','쏘다쏘다','최악','조금','탄맛','미지근'],
                     '아메':['별로','쏘다쏘다','최악','조금','탄맛'],
                     '아아':['별로','쏘다쏘다','최악','조금','탄맛'],
                     '라떼':['별로','쏘다쏘다','최악','조금','탄맛','물맛'],
                     '라뗴':['별로','쏘다쏘다','최악','조금','탄맛'],
                     '라테':['연해'],
                     '맛':['별로','최악','심해','심하','약하다','떨어지다'],
                     '케익':['딱딱','퍽퍽','별로'],
                     '케잌':['딱딱','퍽퍽','별로'],
                     '케이크':['딱딱','퍽퍽','별로'],
                     '마카롱':['딱딱','퍽퍽','별로'],
                     '메뉴':['최악','별로','다시는'],
                     '샌드위치':[],
                     '자몽':[],
                      '스콘':[],
                      '돌체':[],
                     '기대':['이하']}

taste_good_emotion = ['굿또','옳다','사랑','달콤', '깔끔', '신선', '정갈', '싱싱', '적절', '달달', '부드럽', '환상', '담백', '상큼', '맛있다', '부드', '촉촉', '고소','맛나','맛집','존맛','쫀맛']

taste_bad_emotion = ['오래되다','타다맛', '맹맹', '딱딱', '차갑다', '느끼다하다', '심하', '탄맛', '밍밍', '쏘다쏘다', '싱거', '싱겁', '유통기한', '조화', '짜다', '퍽퍽별로','노맛']



negative_word_emotion = ['안','않','없','아닌','아니']

In [ ]:
def get_feature_keywords(feature_keywords, review):
    feature_temp = []
    for keyword in feature_keywords:
        if re.findall(keyword, review):
                
            # a = re.findall(keyword +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # K한 한 한글
            # b = re.findall(keyword + '+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # K 한 한글 
            # c = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + keyword +'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # 한 K한글 예쁜 분위기가
            d = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z| ]?' + keyword +'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z| ]?',review)
                
            # for ngram in a:
            #     t = ()
            #     feature_temp.append(t + (ngram,keyword))
            # for ngram in b:
            #     t = ()
            #     feature_temp.append(t + (ngram,keyword))
            # for ngram in c:
            #     t = ()
            #     feature_temp.append(t + (ngram,keyword))
            for ngram in d:
                t = ()
                feature_temp.append(t + (ngram,keyword))
                
    return feature_temp

In [ ]:
def get_feature_emotions(feature_good_dict,feature_bad_dict, feature_temp):
    good_feature_emotion_list = []
    bad_feature_emotion_list = []
    # last = ''
    for ngrams in feature_temp:
        keyword = ngrams[1]
        # if last == keyword:
        #     continue
        ngram = ngrams[0]
        is_bad_feature = None
        
        good_emotion_list = feature_good_dict[keyword]
        bad_emotion_list = feature_bad_dict[keyword]
        emotion_negative_check = ''
        for emotion in good_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = False
                emotion_negative_check = emotion
        for emotion in bad_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = True
                emotion_negative_check = emotion
        for negative in negative_word_emotion:
            if re.findall(negative, ngram):
                if is_bad_feature == True:
                    if negative in emotion_negative_check:
                        is_bad_feature == True
                        break
                    else:
                        is_bad_feature = False
                        break
                elif is_bad_feature == False:
                    if negative in emotion_negative_check:
                        is_bad_feature = False
                        break
                    else:
                        is_bad_feature = True
                        break
                else:
                    is_bad_feature = True
                    break   
        if is_bad_feature:
            bad_feature_emotion_list.append(ngram)
        elif is_bad_feature == False:
            good_feature_emotion_list.append(ngram)
        else:
            pass
        # last = keyword
    return good_feature_emotion_list, bad_feature_emotion_list

In [ ]:
def get_service_emotion(service_good_emotions, service_bad_emotions):
    bad_service_emotion_list = []
    good_service_emotion_list = []
    for ngrams in service_good_emotions:
        ngram = ngrams[0]
        is_bad_service = False
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_service = True
        if is_bad_service:
            bad_service_emotion_list.append(ngram)
        else:
            good_service_emotion_list.append(ngram)
    
    for ngrams in service_bad_emotions:
        ngram = ngrams[0]
        is_bad_service = True
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_service = False
        if is_bad_service:
            bad_service_emotion_list.append(ngram)
        else:
            good_service_emotion_list.append(ngram)
            
    return good_service_emotion_list, bad_service_emotion_list


In [ ]:
def get_atmosphere_emotion(atmosphere_good_emotions, atmosphere_bad_emotions):
    bad_atmosphere_emotion_list = []
    good_atmosphere_emotion_list = []
    for ngrams in atmosphere_good_emotions:
        ngram = ngrams[0]
        is_bad_atmosphere = False
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_atmosphere = True
        if is_bad_atmosphere:
            bad_atmosphere_emotion_list.append(ngram)
        else:
            good_atmosphere_emotion_list.append(ngram)
    
    for ngrams in atmosphere_bad_emotions:
        ngram = ngrams[0]
        is_bad_atmosphere = True
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_atmosphere = False
        if is_bad_atmosphere:
            bad_atmosphere_emotion_list.append(ngram)
        else:
            good_atmosphere_emotion_list.append(ngram)
            
    return good_atmosphere_emotion_list, bad_atmosphere_emotion_list

In [ ]:
def get_convenient_emotion(convenient_good_emotions, convenient_bad_emotions):
    bad_convenient_emotion_list = []
    good_convenient_emotion_list = []
    for ngrams in convenient_good_emotions:
        ngram = ngrams[0]
        is_bad_convenient = False
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_convenient = True
        if is_bad_convenient:
            bad_convenient_emotion_list.append(ngram)
        else:
            good_convenient_emotion_list.append(ngram)
    
    for ngrams in convenient_bad_emotions:
        ngram = ngrams[0]
        is_bad_convenient = True
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_convenient = False
        if is_bad_convenient:
            bad_convenient_emotion_list.append(ngram)
        else:
            good_convenient_emotion_list.append(ngram)
            
    return good_convenient_emotion_list, bad_convenient_emotion_list

In [ ]:
def get_taste_emotion(taste_good_emotions, taste_bad_emotions):
    bad_taste_emotion_list = []
    good_taste_emotion_list = []
    for ngrams in taste_good_emotions:
        ngram = ngrams[0]
        is_bad_taste = False
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_taste = True
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
    
    for ngrams in taste_bad_emotions:
        ngram = ngrams[0]
        is_bad_taste = True
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_taste = False
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
            
    return good_taste_emotion_list, bad_taste_emotion_list


In [ ]:
def do_stemming(reviews):
    result_list = []
    for i in range(len(reviews)):
        a = re.sub(' ','s',reviews[i])    #띄어쓰기 위해 s 붙힘
        b = twitter.pos(a, stem = True, norm= True)  #형태소 분석
        string = ''
        for word in b:                #문장으로 합침
            string += word[0]
        c = re.sub('s',' ',string)    #띄어쓰기 분리
        result_list.append(c)
    return result_list

In [ ]:
review = pd.read_csv('/content/drive/MyDrive/미디어젠/bert_pred_labels_final.csv')
review.rename(
    columns=({ 'Unnamed: 0': 'id'}), 
    inplace=True,
)
review1 = review.copy()

In [ ]:
def store_name_scaling(storename):
  if storename[-1] != '점':
    storename = storename + '점'
  return storename

In [ ]:
review1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179216 entries, 0 to 179215
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   매장      179216 non-null  object 
 1   시간      179216 non-null  object 
 2   별점      179216 non-null  float64
 3   리뷰      179216 non-null  object 
 4   싸이트     179216 non-null  int64  
 5   감정      179216 non-null  object 
 6   pred    179216 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 9.6+ MB


In [ ]:
review1['매장'] = review1.apply(lambda x: store_name_scaling(x['매장']), axis = 1)

In [ ]:
# review1['매장'].unique()

In [ ]:
review1.sort_values(by=['매장'], axis=0, inplace=True)
review1.reset_index(drop=False,inplace=True)

In [ ]:
reviewlist = []
reviewlist.append(review1.to_dict())
# reviewlist

In [ ]:
for i, reviews in enumerate(reviewlist[:30]):
  stores = reviews['매장']

In [ ]:
# for reviews in reviewlist:
#     # print(reviews['store'])
#     reviews_list = do_stemming(reviews['리뷰'])
#     for i, review in enumerate(reviews_list):
#         # print(store)
#         print(review)
#         service_temp = get_feature_keywords(service_good_feature.keys(), review)
#         good_service,bad_service = get_feature_emotions(service_good_feature, service_bad_feature, service_temp)
#         service_good_emotion_temp = get_feature_keywords(service_good_emotion, review)
#         service_bad_emotion_temp = get_feature_keywords(service_bad_emotion, review)
#         good_service2, bad_service2 = get_service_emotion(service_good_emotion_temp,service_bad_emotion_temp)
#         good_service.extend(good_service2)
#         bad_service.extend(bad_service2)

#         atmosphere_temp = get_feature_keywords(atmosphere_good_feature.keys(), review)
#         good_atmosphere,bad_atmosphere = get_feature_emotions(atmosphere_good_feature, atmosphere_bad_feature, atmosphere_temp)
#         atmosphere_good_emotion_temp = get_feature_keywords(atmosphere_good_emotion, review)
#         atmosphere_bad_emotion_temp = get_feature_keywords(atmosphere_bad_emotion, review)
#         good_atmosphere2, bad_atmosphere2 = get_atmosphere_emotion(atmosphere_good_emotion_temp,atmosphere_bad_emotion_temp)
#         good_atmosphere.extend(good_atmosphere2)
#         bad_atmosphere.extend(bad_atmosphere2)

#         convenient_temp = get_feature_keywords(convenient_good_feature.keys(), review)
#         good_convenient,bad_convenient = get_feature_emotions(convenient_good_feature, convenient_bad_feature, convenient_temp)
#         convenient_good_emotion_temp = get_feature_keywords(convenient_good_emotion, review)
#         convenient_bad_emotion_temp = get_feature_keywords(convenient_bad_emotion, review)


#         # cost_temp = get_feature_keywords(cost_good_feature.keys(), review)
#         # good_cost,bad_cost = get_feature_emotions(cost_good_feature, cost_bad_feature, cost_temp)

#         # visit_temp = get_feature_keywords(visit_good_feature.keys(), review)
#         # good_visit,bad_visit = get_feature_emotions(visit_good_feature, visit_bad_feature, visit_temp)

#         taste_temp = get_feature_keywords(taste_good_feature.keys(), review)
#         good_taste,bad_taste = get_feature_emotions(taste_good_feature, taste_bad_feature, taste_temp)
#         taste_good_emotion_temp = get_feature_keywords(taste_good_emotion, review)
#         taste_bad_emotion_temp = get_feature_keywords(taste_bad_emotion, review)
#         good_taste2, bad_taste2 = get_taste_emotion(taste_good_emotion_temp,taste_bad_emotion_temp)
#         good_taste.extend(good_taste2)
#         bad_taste.extend(bad_taste2)

#         # print(i)
#         good_service = set(good_service)
#         good_service = list(good_service)
#         gsl = len(good_service)
#         for ngram in good_service:
#             ngram = ngram.strip()
#             if '불친' in ngram:
#                 gsl = gsl - 1
#                 continue
#             else:
#                 pass
#             print('good service emotions :{}'.format(gsl))
#             print(ngram)

#         bad_service = set(bad_service)
#         bad_service = list(bad_service)
#         print('bad service emotions :{}'.format(len(bad_service)))
#         for ngram in bad_service:
#             print(ngram)

#         good_atmosphere = set(good_atmosphere)
#         good_atmosphere = list(good_atmosphere)
#         print('good atmosphere emotions :{}'.format(len(good_atmosphere)))
#         for ngram in good_atmosphere:
#             print(ngram)

#         bad_atmosphere = set(bad_atmosphere)
#         bad_atmosphere = list(bad_atmosphere)
#         print('bad atmosphere emotions :{}'.format(len(bad_atmosphere)))
#         for ngram in bad_atmosphere:
#             print(ngram)

#         good_convenient = set(good_convenient)
#         good_convenient = list(good_convenient)
#         print('good convenient emotions :{}'.format(len(good_convenient)))
#         for ngram in good_convenient:
#             print(ngram)

#         bad_convenient = set(bad_convenient)
#         bad_convenient = list(bad_convenient)
#         print('bad convenient emotions :{}'.format(len(bad_convenient)))
#         for ngram in bad_convenient:
#             print(ngram)

#         good_taste = set(good_taste)
#         good_taste = list(good_taste)
#         print('good taste emotions :{}'.format(len(good_taste)))
#         for ngram in good_taste:
#             print(ngram)

#         bad_taste = set(bad_taste)
#         bad_taste = list(bad_taste)
#         print('bad taste emotions :{}'.format(len(bad_taste)))
#         for ngram in bad_taste:
#             print(ngram)

#             convenient_good_feature

In [48]:
check_division = lambda x, y: y if y ==0 else round((x / float(y)),2)

restaurant_good_service_count = 0
restaurant_bad_service_count = 0
restaurant_good_atmosphere_count =0
restaurant_bad_atmosphere_count =0
restaurant_good_taste_count = 0
restaurant_bad_taste_count = 0

with open("/content/drive/MyDrive/미디어젠/감성사전_가게별_점수.txt", "w") as f:
  f.write('Store\tReview_num\tService_Review_num\tGood_service\tBad_service\tAtm_Review_num\tGood_atm\tBad_atm\tTaste_Review_num\tGood_taste\tBad_taste\n')

for i, reviews in enumerate(reviewlist[:30]):
  reviews_list = do_stemming(reviews['리뷰'])
  last = ''
  review_num = 0
  for i, review in enumerate(reviews_list):
      if stores[i] == last: # 계속 같은 지점이라면 이어서 score 연산
        pass
      elif last == '':
        pass
      elif last != '' and stores[i] != last: # 이전과 다른 지점이라면 새로 score 연산

        print(last)
        print('Total review count: {}'.format(review_num))
        print('Good service: {}/{} = {}'.format(restaurant_good_service_count,restaurant_good_service_count + restaurant_bad_service_count,check_division(restaurant_good_service_count, restaurant_good_service_count + restaurant_bad_service_count)))
        print('Bad service: {}/{} = {}'.format(restaurant_bad_service_count,restaurant_good_service_count + restaurant_bad_service_count, check_division(restaurant_bad_service_count, restaurant_good_service_count + restaurant_bad_service_count)))
        print('Good atmosphere: {}/{} = {}'.format(restaurant_good_atmosphere_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count,check_division(restaurant_good_atmosphere_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count)))
        print('Bad atmosphere: {}/{} = {}'.format(restaurant_bad_atmosphere_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count, check_division(restaurant_bad_atmosphere_count ,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count)))
        print('Good taste: {}/{} = {}'.format(restaurant_good_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count,check_division(restaurant_good_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count)))
        print('Bad taste: {}/{} = {}'.format(restaurant_bad_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count,check_division(restaurant_bad_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count)))

        #tsv 형태로 file 저장
        with open("/content/drive/MyDrive/미디어젠/감성사전_가게별_점수.txt", "a") as f:
          f.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(
              last,review_num,\
              
              restaurant_good_service_count + restaurant_bad_service_count,\
              check_division(restaurant_good_service_count, restaurant_good_service_count + restaurant_bad_service_count),\
              check_division(restaurant_bad_service_count, restaurant_good_service_count + restaurant_bad_service_count),\

              restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count,\
              check_division(restaurant_good_atmosphere_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count),\
              check_division(restaurant_bad_atmosphere_count ,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count),\

              restaurant_good_taste_count + restaurant_bad_taste_count,\
              check_division(restaurant_good_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count),\
              check_division(restaurant_bad_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count)))
          
        restaurant_good_service_count = 0
        restaurant_bad_service_count = 0
        restaurant_good_atmosphere_count =0
        restaurant_bad_atmosphere_count =0
        restaurant_good_taste_count = 0
        restaurant_bad_taste_count = 0

        review_num = 0

      service_temp = get_feature_keywords(service_good_feature.keys(), review)
      good_service,bad_service = get_feature_emotions(service_good_feature, service_bad_feature, service_temp)
      service_good_emotion_temp = get_feature_keywords(service_good_emotion, review)
      service_bad_emotion_temp = get_feature_keywords(service_bad_emotion, review)
      good_service2, bad_service2 = get_service_emotion(service_good_emotion_temp,service_bad_emotion_temp)
      good_service.extend(good_service2)
      bad_service.extend(bad_service2)
      gsl = len(good_service)
      for ngram in good_service:
            ngram = ngram.strip()
            if '불친' in ngram:
                gsl = gsl - 1

      atmosphere_temp = get_feature_keywords(atmosphere_good_feature.keys(), review)
      good_atmosphere,bad_atmosphere = get_feature_emotions(atmosphere_good_feature, atmosphere_bad_feature, atmosphere_temp)
      atmosphere_good_emotion_temp = get_feature_keywords(atmosphere_good_emotion, review)
      atmosphere_bad_emotion_temp = get_feature_keywords(atmosphere_bad_emotion, review)
      good_atmosphere2, bad_atmosphere2 = get_atmosphere_emotion(atmosphere_good_emotion_temp,atmosphere_bad_emotion_temp)
      good_atmosphere.extend(good_atmosphere2)
      bad_atmosphere.extend(bad_atmosphere2)

      taste_temp = get_feature_keywords(taste_good_feature.keys(), review)
      good_taste,bad_taste = get_feature_emotions(taste_good_feature, taste_bad_feature, taste_temp)
      taste_good_emotion_temp = get_feature_keywords(taste_good_emotion, review)
      taste_bad_emotion_temp = get_feature_keywords(taste_bad_emotion, review)
      good_taste2, bad_taste2 = get_taste_emotion(taste_good_emotion_temp,taste_bad_emotion_temp)
      good_taste.extend(good_taste2)
      bad_taste.extend(bad_taste2)

      if len(good_service) > len(bad_service):
        restaurant_good_service_count += 1
      elif len(good_service) < len(bad_service):
        restaurant_bad_service_count += 1
      else:
        pass
          
      if len(good_atmosphere) > len(bad_atmosphere):
        restaurant_good_atmosphere_count += 1
      elif len(good_atmosphere) < len(bad_atmosphere):
        restaurant_bad_atmosphere_count += 1
      else:
        pass
          
      if len(good_taste) > len(bad_taste):
        restaurant_good_taste_count += 1
      elif len(good_taste) < len(bad_taste):
        restaurant_bad_taste_count += 1
      else:
        pass
      
      last = stores[i]
      review_num += 1

스타벅스 GS타워점
Total review count: 253
Good service: 49/49 = 1.0
Bad service: 0/49 = 0.0
Good atmosphere: 6/23 = 0.26
Bad atmosphere: 17/23 = 0.74
Good taste: 31/31 = 1.0
Bad taste: 0/31 = 0.0
스타벅스 IFC몰R점
Total review count: 1
Good service: 0/0 = 0
Bad service: 0/0 = 0
Good atmosphere: 1/1 = 1.0
Bad atmosphere: 0/1 = 0.0
Good taste: 0/0 = 0
Bad taste: 0/0 = 0
스타벅스 SSG마켓도곡R점
Total review count: 273
Good service: 53/55 = 0.96
Bad service: 2/55 = 0.04
Good atmosphere: 19/27 = 0.7
Bad atmosphere: 8/27 = 0.3
Good taste: 42/45 = 0.93
Bad taste: 3/45 = 0.07
스타벅스 W-Mall점
Total review count: 378
Good service: 58/59 = 0.98
Bad service: 1/59 = 0.02
Good atmosphere: 17/25 = 0.68
Bad atmosphere: 8/25 = 0.32
Good taste: 58/58 = 1.0
Bad taste: 0/58 = 0.0
스타벅스 W-mall점
Total review count: 3
Good service: 0/0 = 0
Bad service: 0/0 = 0
Good atmosphere: 0/1 = 0.0
Bad atmosphere: 1/1 = 1.0
Good taste: 0/0 = 0
Bad taste: 0/0 = 0
스타벅스 가든파이브점
Total review count: 231
Good service: 38/38 = 1.0
Bad service: 0/38 = 0.